In [1]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from googleapiclient.discovery import build



In [2]:

model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:

emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
label_encoder = LabelEncoder()
label_encoder.fit(emotion_labels)


LabelEncoder()

In [4]:
def detect_emotion(text):
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True)
    outputs = model(inputs)
    logits = outputs.logits
    predicted_class_id = tf.argmax(logits, axis=1).numpy()[0]
    emotion = label_encoder.inverse_transform([predicted_class_id])[0]
    return emotion


In [ ]:
YOUTUBE_API_KEY = "your_api_key"  

def recommend_youtube_music(emotion):
    youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)

    
    emotion_to_query = {
        "joy": "happy upbeat music",
        "sadness": "soothing emotional piano music",
        "anger": "calming instrumental music",
        "fear": "reassuring ambient music",
        "disgust": "relaxing nature sounds",
        "surprise": "mysterious cinematic soundtrack",
        "neutral": "chill lo-fi beats"
    }

    query = emotion_to_query.get(emotion, "calm instrumental music")

    request = youtube.search().list(
        part="snippet",
        q=query,
        type="video",
        maxResults=3,
        videoCategoryId="10"  
    )

    response = request.execute()
    
    results = []
    for item in response.get("items", []):
        video_id = item["id"]["videoId"]
        title = item["snippet"]["title"]
        url = f"https://www.youtube.com/watch?v={video_id}"
        results.append((title, url))

    return results



In [6]:
def moodmirror(text):
    emotion = detect_emotion(text)
    print(f"Detected emotion: {emotion}")
    
    recommendations = recommend_youtube_music(emotion)
    print("\nRecommended YouTube Music Links:")
    for title, url in recommendations:
        print(f"{title}: {url}")


In [9]:
sample_text = "i feel happy ."
moodmirror(sample_text)


Detected emotion: joy

Recommended YouTube Music Links:
Happy Weekend Beats - Good Vibes Only - Upbeat Music to Be Happy: https://www.youtube.com/watch?v=I35paFqFOPk
Songs that&#39;ll make you dance the whole day ~ Mood booster playlist: https://www.youtube.com/watch?v=ljnGl5nvUJY
Pharrell Williams - Happy (Video): https://www.youtube.com/watch?v=ZbZSe6N_BXs
